In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from wordcloud import WordCloud

#to data preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

#NLP tools
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer

#train split and fit models
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

#model selection
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
import joblib

# Load the dataset
dataset = pd.read_csv('labeled_data.csv')

# Preprocessing function
def preprocess_text(text):
    review = re.sub('[^a-zA-Z]', ' ', text)
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    all_stopwords = stopwords.words('english')
    all_stopwords.remove('not')
    review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
    review = ' '.join(review)
    return review

# Apply preprocessing to the dataset
dataset['clean_text'] = dataset['tweet'].apply(preprocess_text)

# Vectorization
cv = CountVectorizer(max_features=2000)
X = cv.fit_transform(dataset['clean_text']).toarray()
y = dataset['class']

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True)# random_state=0)

# Model Training

# naive-bayes:
classifier_np = GaussianNB()
classifier_np.fit(X_train, y_train)

# xgboost:
classifier_xgb= XGBClassifier()
classifier_xgb.fit(X_train, y_train)

# Dump the trained model and CountVectorizer
joblib.dump(classifier_np, 'NB_classifier.pkl')
joblib.dump(classifier_xgb, 'XGB_classifier.pkl')
joblib.dump(cv, 'count_vectorizer.pkl')

# Later, to load the model and make predictions:

# Load the saved model and CountVectorizer
loaded_model1 = joblib.load('NB_classifier.pkl')
loaded_model2 = joblib.load('XGB_classifier.pkl')
loaded_cv = joblib.load('count_vectorizer.pkl')

# **Testing:**

In [ ]:
# Take user input
user_input = input("Enter a string: ")

# Preprocess the user input
preprocessed_input = preprocess_text(user_input)

# Vectorize the preprocessed input
user_input_vectorized = loaded_cv.transform([preprocessed_input]).toarray()

# Predict
prediction1 = loaded_model1.predict(user_input_vectorized)
prediction2 = loaded_model2.predict(user_input_vectorized)

if prediction1 == 0:
    print("The string is not hate speech.")
else:
    print("The string is hate speech.")

if prediction2 == 0:
    print("The string is not hate speech.")
else:
    print("The string is hate speech.")